<a href="https://colab.research.google.com/github/flari-gold/IU/blob/main/IU_RedditDataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#package installs with pip
!pip install praw
!pip install HanTa

#import modules
from google.colab import userdata
import praw
import collections
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from HanTa import HanoverTagger

In [ ]:
#parameter
#Daten der unter https://old.reddit.com/prefs/apps/ generierten Anwendung. Ausreichend für ReadOnly Zugriffe.
#Die Daten sind im Google Colab Secrets hinterlegt
CLIENT_ID=userdata.get('CLIENT_ID')
CLIENT_SECRET=userdata.get('CLIENT_SECRET')

#Für Write Zugriffe muss ebenfalls noch der eigene Account angegeben werden, kann in diesem Fall ignoriert werden
#USERNAME=userdata.get('USERNAME')
#PASSWORD=userdata.get('PASSWORD')

#Parameter für die Datenanalyse
SUBREDDIT='Sylosis'
ANZAHL_POSTS=100
TOPICS=5

#initialisierung reddit API
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent="Mein Topic Modeling Script", #User Agent ist zwingend notwendig
    check_for_async=False #PRAW wirft eine Warnung welche die ASYNC API empfiehlt, kann ignoriert werden
)

In [ ]:
#declaration
all_textdata = []
user_upvotes = collections.Counter()
user_contributionscounter = collections.Counter()

#Anfragen an die Reddit API mit über request liefern die Daten im JSON Format. PRAW vereinfacht dies, in dem man die direkt die Attribute ansprechen kann
for submission in reddit.subreddit(SUBREDDIT).hot(limit=ANZAHL_POSTS):

    user_contributionscounter[submission.author] += 1
    user_upvotes[submission.author] += submission.score

    post_textdata = [submission.title]#Alle Reddit Beiträge haben einen Titel. Der Titel wird ebenfalls im Topic Modeling wie ein normaler Kommentar behandelt
    post_textdata.append(submission.selftext)#Nicht alle Beträge haben einen Selftext. Beiträge die auf Links verweisen beispielsweise. Falls Selftext vorhanden ist, wird er im Topic Modeling behandelt, ansonsten ist der String leer

#Sobald ein Kommntar auf Reddit zu viele Antworten hat, werden die Antworten hinter einer Schaltfläche "more" verborgen und das gleiche gilt auch beim Zugriff auf die API.
#Das Auflösen der "more" stellt einen eigenen API Aufruf da und sorgt aufgrund der Zugriff Limits bei größeren Anfragen zu einer langen Laufzeit
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        post_textdata.append(comment.body)
        user_contributionscounter[comment.author] += 1
        user_upvotes[comment.author] += comment.score

    all_textdata.append(post_textdata)


print(user_contributionscounter)
print(user_upvotes)



In [ ]:
#clean text
all_cleantext = []

stop_words = stopwords.words('english')
stop_words.extend(['take', 'use', 'would', 'get'])

tagger_en = HanoverTagger.HanoverTagger('morphmodel_en.pgz')
#Vllt noch ändern, Slipknot und Slipknots z.B. sollte schon gelemmat werden
tagger_Exclude = ['sylosis'] #Beim Testen von Subreddits von Musik Bands kommt es mit dem Tagger zu Problemen

for i in all_textdata:
    string_result = []
    for j in i:
        templist = []
        for word in j.lower().split():
          if word not in tagger_Exclude:
            templist.append(tagger_en.analyze(word)[0])
          else:
            templist.append(word)
        templist = [word for word in templist if word not in stop_words]
        tempstring = ' '.join(templist)
        string_result.append(tempstring.replace(".", "").replace(",", "").replace("!", "").replace("?", "").replace("(", "").replace(")", ""))
    all_cleantext.append(string_result)
    #print(string_result)



In [ ]:
from gensim import corpora
from gensim.models import LdaModel

#all_cleantext ist eine Liste, welche Listen beinhaltet, welche wiederum Strings beinhalten
#Jeder String repräsentiert einen Post/Kommentar
#Für die Verarbeitung muss aber in der Liste für jedes Wort ein eigener String stehen
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string.split())

# Create dictionary and corpus
dictionary = corpora.Dictionary(templist)
corpus = [dictionary.doc2bow(text) for text in templist]

# Train LDA model
lda_model = LdaModel(corpus, num_topics=TOPICS, id2word=dictionary, passes=10)

from pprint import pprint
pprint(lda_model.print_topics())


In [ ]:
pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim

# Visualisierung
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(panel)

In [ ]:
import gensim
import gensim.corpora as corpora# Create Dictionary

templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string.split())


id2word = corpora.Dictionary(templist)# Create Corpus
texts = templist# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]# View
#print(corpus[:1][0][:30])

from pprint import pprint
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word, num_topics=TOPICS, passes=10)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]
#print(lda_model.get_topics())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

#all_cleantext ist eine Liste, welche Listen beinhaltet, welche wiederum Strings beinhalten
#Jeder String repräsentiert einen Post/Kommentar
#Für die Verarbeitung muss aber in einer Liste Strings stehen
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string)

# Text data

# Create document-term matrix
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(templist)

# Train LDA model
lda_model = LatentDirichletAllocation(n_components=TOPICS)
lda_model.fit(dtm)

import numpy as np
words = np.array(vectorizer.get_feature_names_out())

# Top-Wörter für jedes Thema
n_top_words = 5
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic #{topic_idx + 1}:")
    print(" ".join(words[np.argsort(topic)[-n_top_words:]]))

#doc_topic_distributions = lda_model.transform(dtm)

# Ausgabe
#for doc_idx, topic_dist in enumerate(doc_topic_distributions):
#    print(f"Document #{doc_idx + 1}:")
#    print(f"Topic Distribution: {topic_dist}")

import pandas as pd

# Top-Wörter pro Thema als DataFrame
topic_word_df = pd.DataFrame(
    lda_model.components_,
    columns=vectorizer.get_feature_names_out()
)
#print(topic_word_df)

doc_topic_df = pd.DataFrame(doc_topic_distributions, columns=[f"Topic {i+1}" for i in range(lda_model.n_components)])
print(doc_topic_df)



In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string)

vectorizer = TfidfVectorizer(use_idf = True, max_features = 5, smooth_idf=True)
#model = vectorizer.fit_transform([templist[0], templist[1], templist[2]])
model = vectorizer.fit_transform(templist)


lda_model=LatentDirichletAllocation(n_components=TOPICS,learning_method='online')
lda_top=lda_model.fit_transform(model)

for i,topic in enumerate(lda_top[0]):
	print("Topic ",i,": ",topic*100,"%")

for i,topic in enumerate(lda_top[1]):
	print("Topic ",i,": ",topic*100,"%")

for i,topic in enumerate(lda_top[2]):
	print("Topic ",i,": ",topic*100,"%")


In [ ]:
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string)
end_string = ' '.join(templist)


print(collections.Counter(end_string.split()).most_common(10))

In [ ]:
templist = []
for i in all_cleantext:
    long_string = ' '.join(i)
    templist.append(long_string)
end_string = ' '.join(templist)
print(end_string)

#
from wordcloud import WordCloud
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')# Generate a word cloud
wordcloud.generate(long_string)# Visualize the word cloud
wordcloud.to_image()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 10, 100)

plt.plot(x, np.sin(x))
plt.plot(x, np.cos(x))

plt.show()

In [ ]:
!pip install HanTa
from HanTa import HanoverTagger

tagger_en = HanoverTagger.HanoverTagger('morphmodel_en.pgz')

print(tagger_en.analyze('driving'))
print(type(tagger_en.analyze('driving')))
print(tagger_en.analyze('driving')[0])